
Naive Bayes - Trabalho - Jader
Questão 1

Implemente um classifacor Naive Bayes para o problema de predizer a qualidade de um carro. Para este fim, utilizaremos um conjunto de dados referente a qualidade de carros, disponível no UCI. Este dataset de carros possui as seguintes features e classe:

Attributos

    buying: vhigh, high, med, low
    maint: vhigh, high, med, low
    doors: 2, 3, 4, 5, more
    persons: 2, 4, more
    lug_boot: small, med, big
    safety: low, med, high

Classes

    unacc, acc, good, vgood


QUESTÃO 1:


In [16]:
import csv
import random
import math
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [27]:
def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [str(x) for x in dataset[i]]
    return dataset

import random

def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated


def mean(numbers):
    return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            if stdev == 0.0:
                stdev = 10000.0
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

def getPredictions(summaries, testSet):
    predictions = []
    for _,row in testSet.iterrows():
        result = predict(summaries, row)
        predictions.append(result)
    return predictions

def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [3]:
ds = loadCsv('carData.csv')

In [25]:
filename = 'carData.csv'
dataset = pd.read_csv(filename)
for i in range(0, dataset.shape[1]):
    dataset.iloc[:,i] = LabelEncoder().fit_transform(dataset.iloc[:,i])
    
Y = dataset.unacc.copy()
del dataset['unacc']
X = dataset

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3)

X_train_sk = X_train.copy()

idx = X_train.shape[1]
new_col = Y_train 
X_train.insert(loc=idx, column='Class', value=new_col)

X_train.head()

,vhigh,vhigh.1,2,2.1,small,low,Class
159,3,0,1,2,0,2,2
1622,1,1,0,0,1,1,2
1032,2,0,2,0,0,2,2
731,0,2,3,0,1,1,2
1272,2,1,3,0,1,2,2


In [26]:
summaries = summarizeByClass(X_train)

KeyError: 0

In [6]:
summaries = summarizeByClass(X_train)
Y_test.head()

[['low', 'vhigh', '5more', '2', 'small', 'med', 'unacc'], ['med', 'low', '4', '2', 'med', 'med', 'unacc'], ['vhigh', 'med', '5more', '4', 'big', 'high', 'acc'], ['vhigh', 'vhigh', '2', '4', 'med', 'low', 'unacc'], ['med', 'med', '2', '4', 'big', 'low', 'unacc'], ['low', 'vhigh', '4', '2', 'small', 'high', 'unacc'], ['low', 'high', '3', 'more', 'small', 'low', 'unacc'], ['high', 'high', '3', 'more', 'small', 'high', 'acc'], ['vhigh', 'high', '3', '2', 'med', 'low', 'unacc'], ['low', 'low', '3', '2', 'big', 'high', 'unacc'], ['low', 'vhigh', '5more', 'more', 'med', 'med', 'acc'], ['vhigh', 'vhigh', '2', '2', 'big', 'high', 'unacc'], ['vhigh', 'vhigh', '3', '2', 'big', 'high', 'unacc'], ['med', 'high', '3', 'more', 'big', 'high', 'acc'], ['high', 'med', '2', '2', 'big', 'med', 'unacc'], ['high', 'vhigh', '5more', '4', 'big', 'low', 'unacc'], ['low', 'low', '3', '4', 'small', 'high', 'good'], ['med', 'med', '5more', '4', 'big', 'high', 'vgood'], ['low', 'med', '3', '2', 'med', 'high', 'una

QUESTÃO 2:

In [7]:
from sklearn.naive_bayes import GaussianNB
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [8]:
df = pd.read_csv('carData.csv', names=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'classe'])

df['buying'] = LabelEncoder().fit(df.buying).transform(df.buying)
df['maint'] = LabelEncoder().fit(df.maint).transform(df.maint)
df['doors'] = LabelEncoder().fit(df.doors).transform(df.doors)
df['persons'] = LabelEncoder().fit(df.persons).transform(df.persons)
df['lug_boot'] = LabelEncoder().fit(df.lug_boot).transform(df.lug_boot)
df['safety'] = LabelEncoder().fit(df.safety).transform(df.safety)
df['classe'] = LabelEncoder().fit(df.classe).transform(df.classe)

df.head()

,buying,maint,doors,persons,lug_boot,safety,classe
0,3,3,0,0,2,1,2
1,3,3,0,0,2,2,2
2,3,3,0,0,2,0,2
3,3,3,0,0,1,1,2
4,3,3,0,0,1,2,2


In [9]:
features = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
X = df.get(features)
Y = df['classe'].values

X = MinMaxScaler().fit_transform(df[features])

In [11]:
gnb = GaussianNB()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state=1)
y_pred = gnb.fit(X_train, Y_train).predict(X_test)
accuracy_score(Y_test, y_pred)

0.65740740740740744

In [12]:
print(classification_report(y_pred, Y_test))

             precision    recall  f1-score   support

          0       0.11      0.67      0.19        15
          1       0.00      0.00      0.00         0
          2       0.83      0.85      0.84       305
          3       1.00      0.14      0.25       112

avg / total       0.85      0.66      0.66       432



/home/jader/miniconda3/envs/aula/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
